# Physics-Informed Neural Networks for Burgers Equation
## A Tutorial on Implementing Viscous and Inviscid Solutions

In this tutorial, we'll explore Physics-Informed Neural Networks (PINNs) by implementing solvers for both the viscous and inviscid Burgers equation. The tutorial will guide you through:

1. Setting up a PINN architecture
2. Implementing the viscous Burgers equation (provided)
3. Implementing the inviscid Burgers equation (exercise)
4. Comparing and analyzing the results

The Burgers equation is a fundamental PDE that appears in various fields of fluid mechanics and acoustics. We'll study both versions:

- Viscous: $\frac{\partial u}{\partial t} + u\frac{\partial u}{\partial x} = \nu\frac{\partial^2 u}{\partial x^2}$
- Inviscid: $\frac{\partial u}{\partial t} + u\frac{\partial u}{\partial x} = 0$

where $\nu$ is the viscosity coefficient.

## 1. Introduction to vanilla PINNs <a name="1-bullet"></a>

First, let us recall the general formulation of the vanilla PINNs. Considering the following parameterized PDE defined on the domain $\Omega \subset \mathbb{R}^d$ with the boundary $\partial\Omega$:
\begin{align}
&\partial_t \boldsymbol{u} + N_{\boldsymbol{\mathrm{x}}}(\boldsymbol{u}, \lambda)=0, \text{ for } \boldsymbol{\mathrm{x}}\in\Omega, t\in[0,T]\\
&\boldsymbol{u}(\boldsymbol{\mathrm{x}},0)=g(\boldsymbol{\mathrm{x}}), \text{ for } \boldsymbol{\mathrm{x}}\in\Omega\\
&B(\boldsymbol{u},\boldsymbol{\mathrm{x}},t)=0, \text{ for } \boldsymbol{\mathrm{x}}\in\partial\Omega\\
\end{align}
where $\boldsymbol{\mathrm{x}} \in \mathbb{R}^d$ and $t$ are the spatial and temporal coordinates, $N_{\boldsymbol{\mathrm{x}}}$ is a differential operator, $\boldsymbol{\lambda}$ is the PDE parameter, $\boldsymbol{u}$ is the solution of the PDE with initial condition $g(\boldsymbol{\mathrm{x}})$ and boundary condition $B$, which could be Dirichlet, Neumann, Robin, or periodic boundary conditions. The subscripts denote the partial differentiation in time or space. In PINNs, the solution $\boldsymbol{u}$ of the PDE is approximated by a fully-connected feedforward neural network $\Phi$:
\begin{align*}
    \boldsymbol{u} \approx \hat{\boldsymbol{u}} = \Phi(\boldsymbol{\mathrm{x}}, t, \boldsymbol{\theta})
\end{align*}
where $\hat{\boldsymbol{u}}$ denotes the prediction value for the solution and $\boldsymbol{\theta}$ denotes the trainable parameters of the neural network. The parameters of the neural network are trained by minimizing the cost function $L$:
\begin{align}
    L = L_{pde} + L_{ic} + L_{bc} + L_{data}
\end{align}
where the terms $L_{pde},L_{ic},L_{bc}$ and $L_{data}$ penalize the loss in the residual of the PDE, the initial condition, the boundary condition, and the supervised data (measurements), respectively, which can be represented as follow:
\begin{align*}
    L_{pde} &= \dfrac{1}{N_{pde}}\sum_{i=1}^{N_{pde}}|\hat{\boldsymbol{u}}_{t^i} + \Phi_{\boldsymbol{\mathrm{x}}^{i}}(\hat{\boldsymbol{u}^{i}}, \boldsymbol{\lambda})|^2\\
    L_{ic} &= \dfrac{w_{ic}}{N_{ic}}\sum_{i=1}^{N_{ic}}|\boldsymbol{\hat{u}}(\boldsymbol{\mathrm{x}}^i,0) - g(\boldsymbol{\mathrm{x}}^i)|^2\\
    L_{bc} &= \dfrac{w_{bc}}{N_{bc}}\sum_{i=1}^{N_{bc}}|B(\hat{\boldsymbol{u}^i}, \boldsymbol{\mathrm{x}}^{i}, t^{i})|^2\\
    L_{data} &= \dfrac{w_{data}}{N_{data}}\sum_{i=1}^{N_{data}}|\hat{\boldsymbol{u}}(\boldsymbol{\mathrm{x}}^i,t^i) - \boldsymbol{u}(\boldsymbol{\mathrm{x}}^i,t^i)|^2\\
\end{align*}
where $w_{ic}, w_{bc}, w_{data}$ are the weights to balance different terms in the cost function.

The definition of the cost function $L$ and the trainable parameters depend on the problems:

|Type of problem | Input of $\Phi$ | Trainable parameter | Loss function |
| :- | -: | -: |-: |
| Forward | $(\boldsymbol{\mathrm{x}},t)$ | $\theta$ | $L = L_{pde} + L_{ic} + L_{bc}$
| Inverse | $(\boldsymbol{\mathrm{x}},t)$ | $\theta,\lambda$ | $L = L_{pde} + L_{data}$
| Ill-posed | $(\boldsymbol{\mathrm{x}},t)$ | $\theta$ | $L = L_{pde} + L_{data}$
| Generalization | $(\boldsymbol{\mathrm{x}},t,\lambda)$ | $\theta$ | $L = L_{pde} + L_{ic} + L_{bc}$


## Forward problem
In forward problems, the initial and/or boundary conditions (IC and/or BCs) of the PDEs are well-defined. The measurements of the solutions (supervised data) are optional. We aim to infer the solution in the entire domain using PINNs.

To solve a PDE problem, there exists two equivalent formulations: the strong form and the weak form.
* The strong form consists of the differential equations along with IC and BCs. It imposes continuity and differentiability requirements on the potential solutions to the equation.
* The weak form is an alternative representation of the differential equations, which relaxes the continuity and differentiability requirements. It reduces the order of the derivatives and forces the solution to satisfied an integral functions.

### Example: Viscous Burgers equation <a name="1.1-bullet"></a>
We consider the following viscous Burgers equation:
\begin{align}
  &u_t + uu_x -\nu u_{xx} = 0 \quad \text{for } x\in [0,2], t\in[0, 1] \\
  &u(x,0) = -\sin(\pi x) \quad \text{for } x\in [0,2]\\
  &u(0,t) = u(2,t) = 0 \quad \text{for } t\in [0,1]
\end{align}
where $\nu$ is the PDE parameter. In this section, we take $\nu=0.025$.

We denote $\hat{u}=\Phi(\boldsymbol{\mathrm{x}}, t, \boldsymbol{\theta})$ the prediction of PINNs for the solution. The loss function, which includes the loss the IC/BC and the PDE residuals, now becomes:
\begin{align}
  L &= L_{pde} + L_{ic} + L_{bc}\\
    &= \dfrac{1}{N_{pde}}\sum_{i=1}^{N_{pde}}|\hat{u}^i_{t} + \hat{u}^i\hat{u}^i_{x}-\nu \hat{u}_{xx}|^2 + \dfrac{w_{ic}}{N_{ic}}\sum_{i=1}^{N_{ic}}|\hat{u}(x^i,0) + \sin(\pi x^i)|^2 + \dfrac{w_{bc}}{N_{bc}}\sum_{i=1}^{N_{bc}}(|\hat{u}(-1,t^i)|^2+|\hat{u}(1,t^i)|^2)
\end{align}
Here we will take $w_{ic}=w_{bc}=1$.

We take the following steps to train PINNs:
* Define the domain
* Define the initial/boundary conditions (IC/BC) and the training points
* Define PDE residuals and loss term for the PDE
* Define PINNs architecture
* Train PINNs

#### Define the domain

## 2. Setting Up the Environment

First, let's import the required libraries and set up our computational environment. We'll use PyTorch for the neural network implementation, NumPy for numerical operations, and Matplotlib for visualization.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import ExponentialLR

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Parameters for both viscous and inviscid cases
x_min, x_max = 0, 2     # Spatial domain
t_min, t_max = 0, 0.48  # Time domain
viscosity = 0.01/np.pi  # Viscosity coefficient (will be 0 for inviscid case)

# Create directory for saving plots
import os
os.makedirs('./plots', exist_ok=True)

## 3. Neural Network Architecture

We'll use a fully connected neural network with multiple hidden layers. This architecture will be used for both the viscous and inviscid cases. The network takes (x, t) as input and predicts u(x, t) as output.

In [ ]:
class BurgersPINN(nn.Module):
    def __init__(self, layers=[2, 32, 128, 16, 128, 32, 1]):
        super().__init__()

        # Network layers
        self.layers = nn.ModuleList()
        for i in range(len(layers)-1):
            self.layers.append(nn.Linear(layers[i], layers[i+1]))

        # Initialize weights with Xavier
        for layer in self.layers:
            nn.init.xavier_normal_(layer.weight)
            nn.init.zeros_(layer.bias)

        self.activation = nn.Tanh()

    def forward(self, x):
        if not torch.is_tensor(x):
            x = torch.tensor(x, dtype=torch.float32)

        for i in range(len(self.layers)-1):
            x = self.activation(self.layers[i](x))
        x = self.layers[-1](x)
        return x

# Test the network
network = BurgersPINN()
print(f"Network architecture:\n{network}")

## 4. Viscous Burgers Equation Implementation (Reference)

Here's the complete implementation of the PINN solver for the viscous Burgers equation. Study this implementation carefully as you'll need to modify it for the inviscid case.

In [ ]:
# Load reference data from FD solution
fd_data = np.load('burgers_fd_data_viscous.npz')
x_ref = fd_data['x']
t_ref = fd_data['t']
u_ref = fd_data['u']

def predict_solution(solver, x, t):
    """Generate predictions for given space-time points."""
    if not torch.is_tensor(x):
        x = torch.tensor(x, dtype=torch.float32)
    if not torch.is_tensor(t):
        t = torch.tensor(t, dtype=torch.float32)

    x = x.reshape(-1, 1).to(solver.device)
    t = t.reshape(-1, 1).to(solver.device)

    X, T = torch.meshgrid(x.squeeze(), t.squeeze(), indexing='ij')
    X_pred = torch.stack([X.flatten(), T.flatten()], dim=1).to(solver.device)

    solver.network.eval()
    with torch.no_grad():
        u_pred = solver.network(X_pred)

    return u_pred.reshape(x.shape[0], t.shape[0]).cpu().numpy()

def plot_comparison(x, t, u_pinn, x_ref, t_ref, u_ref, case="viscous"):
    """Plot PINN solution against reference data with error analysis."""
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 6))

    # PINN solution
    X, T = np.meshgrid(t, x)
    c1 = ax1.contourf(T, X, u_pinn, levels=50, cmap='rainbow')
    plt.colorbar(c1, ax=ax1)
    ax1.set_xlabel('t')
    ax1.set_ylabel('x')
    ax1.set_title('PINN Solution')

    # Reference solution
    X_ref, T_ref = np.meshgrid(t_ref, x_ref)
    c2 = ax2.contourf(T_ref, X_ref, u_ref, levels=50, cmap='rainbow')
    plt.colorbar(c2, ax=ax2)
    ax2.set_xlabel('t')
    ax2.set_ylabel('x')
    ax2.set_title('Reference Solution (FD)')

    # Error plot
    # Interpolate reference solution to match PINN grid points
    from scipy.interpolate import griddata
    points = np.column_stack((T_ref.flatten(), X_ref.flatten()))
    u_ref_interp = griddata(points, u_ref.flatten(), (T, X), method='cubic')

    error = np.abs(u_pinn - u_ref_interp)
    c3 = ax3.contourf(T, X, error, levels=50, cmap='hot')
    plt.colorbar(c3, ax=ax3)
    ax3.set_xlabel('t')
    ax3.set_ylabel('x')
    ax3.set_title(f'Absolute Error (Max: {np.nanmax(error):.3e})')

    plt.tight_layout()
    plt.savefig(f'./plots/comparison_with_reference_{case}.png')
    plt.show()

    # Plot time slices
    plt.figure(figsize=(15, 5))
    times = [0.0, 0.25, 0.48]
    time_indices = [np.abs(t - time).argmin() for time in times]
    time_indices_ref = [np.abs(t_ref - time).argmin() for time in times]

    for i, (time, idx, idx_ref) in enumerate(zip(times, time_indices, time_indices_ref)):
        plt.subplot(1, 3, i+1)
        plt.plot(x, u_pinn[:, idx], 'r-', label='PINN')
        plt.plot(x_ref, u_ref[:, idx_ref], 'b--', label='Reference')
        plt.xlabel('x')
        plt.ylabel('u')
        plt.title(f't = {time:.2f}')
        plt.grid(True)
        plt.legend()

    plt.tight_layout()
    plt.savefig(f'./plots/comparison_slices_with_reference_{case}.png')
    plt.show()

### Define PINN for the Viscous problem

In [ ]:
class ViscousBurgersSolver:
    def __init__(self, network, device='cpu'):
        self.network = network.to(device)
        self.device = device

        # Generate training points
        self.generate_training_data()

        # Move data to device
        self.X_ic = self.X_ic.to(device)
        self.u_ic = self.u_ic.to(device)
        self.X_bc_left = self.X_bc_left.to(device)
        self.X_bc_right = self.X_bc_right.to(device)
        self.X_colloc = self.X_colloc.to(device)

    def generate_training_data(self):
        # Initial condition points: u(x,0) = sin(πx)
        x_ic = torch.linspace(x_min, x_max, 100).reshape(-1, 1)
        t_ic = torch.zeros_like(x_ic)
        self.X_ic = torch.cat((x_ic, t_ic), dim=1)
        self.u_ic = torch.sin(np.pi * x_ic)

        # Boundary condition points
        t_bc = torch.linspace(t_min, t_max, 100).reshape(-1, 1)
        x_bc_left = torch.zeros_like(t_bc)
        x_bc_right = torch.ones_like(t_bc) * x_max
        self.X_bc_left = torch.cat((x_bc_left, t_bc), dim=1)
        self.X_bc_right = torch.cat((x_bc_right, t_bc), dim=1)

        # Collocation points for PDE
        n_colloc = 10000
        x_colloc = torch.rand(n_colloc, 1) * (x_max - x_min) + x_min
        t_colloc = torch.rand(n_colloc, 1) * (t_max - t_min) + t_min
        self.X_colloc = torch.cat((x_colloc, t_colloc), dim=1)

    def compute_pde_residual(self, x_colloc):
        x_colloc.requires_grad = True
        u = self.network(x_colloc)

        # Calculate derivatives
        u_grad = torch.autograd.grad(u.sum(), x_colloc, create_graph=True)[0]
        u_t = u_grad[:, 1:2]  # ∂u/∂t
        u_x = u_grad[:, 0:1]  # ∂u/∂x

        # Second derivative ∂²u/∂x²
        u_xx = torch.autograd.grad(u_x.sum(), x_colloc, create_graph=True)[0][:, 0:1]

        # PDE residual: ∂u/∂t + u*∂u/∂x - ν*∂²u/∂x²
        residual = u_t + u * u_x - viscosity * u_xx

        return residual

    def loss_ic(self, x, u):
        u_pred = self.network(x)
        return torch.mean((u_pred - u) ** 2)

    def loss_bc(self, x):
        u_pred = self.network(x)
        return torch.mean(u_pred ** 2)

    def loss_pde(self, x):
        residual = self.compute_pde_residual(x)
        return torch.mean(residual ** 2)

    def train(self, epochs=10000, learning_rate=0.001):
        optimizer = torch.optim.Adam(self.network.parameters(), lr=learning_rate)
        scheduler = ExponentialLR(optimizer, gamma=0.9999)

        history = {'total_loss': [], 'ic_loss': [], 'bc_loss': [], 'pde_loss': []}

        for epoch in range(epochs):
            optimizer.zero_grad()

            # Compute losses
            ic_loss = self.loss_ic(self.X_ic, self.u_ic)
            bc_loss = self.loss_bc(self.X_bc_left) + self.loss_bc(self.X_bc_right)
            pde_loss = self.loss_pde(self.X_colloc)

            # Total loss
            total_loss = ic_loss + bc_loss + pde_loss

            # Backpropagation
            total_loss.backward()
            optimizer.step()
            scheduler.step()

            # Store losses
            if epoch % 1000 == 0:
                history['total_loss'].append(total_loss.item())
                history['ic_loss'].append(ic_loss.item())
                history['bc_loss'].append(bc_loss.item())
                history['pde_loss'].append(pde_loss.item())
                print(f"Epoch {epoch}: Loss = {total_loss.item():.6f}")

        return history

### Train PINN for Viscous problem

In [ ]:
# Train and plot the Viscous Burgers Equation solution
x = np.linspace(x_min, x_max, 100)
t = np.linspace(t_min, t_max, 100)

# Train the model
network_viscous = BurgersPINN().to(device)
solver_viscous = ViscousBurgersSolver(network_viscous, device)
history_viscous = solver_viscous.train()

# Get PINN solution
u_pinn = predict_solution(solver_viscous, x, t)

### Plot results

In [ ]:
# Compare with reference solution
plot_comparison(x, t, u_pinn, x_ref, t_ref, u_ref)

# Plot training history
plt.figure(figsize=(10, 5))
plt.semilogy(range(0, len(history_viscous['total_loss'])*100, 100), history_viscous['total_loss'], label='Total Loss')
plt.semilogy(range(0, len(history_viscous['ic_loss'])*100, 100), history_viscous['ic_loss'], label='IC Loss')
plt.semilogy(range(0, len(history_viscous['bc_loss'])*100, 100), history_viscous['bc_loss'], label='BC Loss')
plt.semilogy(range(0, len(history_viscous['pde_loss'])*100, 100), history_viscous['pde_loss'], label='PDE Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training History')
plt.legend()
plt.grid(True)
plt.savefig('./plots/training_history.png')
plt.show()

### Forcing hardly the IC/BC to increase the performance
We recall the initial/boundary conditions (IC/BC) of this problem:
\begin{align}
  &u(x,0) = \sin(\pi x) \quad \text{for } x\in [0,2]\\
  &u(0,t) = u(2,t) = 0 \quad \text{for } t\in [0,1]
\end{align}

We can force the IC/BC to be automatically satisfied. Instead of imposing the prediction $\hat{\boldsymbol{u}} = \Phi(\boldsymbol{\mathrm{x}}, t, \boldsymbol{\theta})$, we can use the following representation:
$$\hat{\boldsymbol{u}} = t(x-0)(x-2) \Phi(\boldsymbol{\mathrm{x}}, t, \boldsymbol{\theta}) + \sin(\pi x)$$
We can do this by creating a new class that defines our problem, `ViscousBurgersSolverBC`.

In [ ]:
class ViscousBurgersSolverBC:
    def __init__(self, network, device='cpu'):
        self.network = network.to(device)
        self.device = device

        # Generate training points
        self.generate_training_data()

        # Move data to device
        self.X_ic = self.X_ic.to(device)
        self.u_ic = self.u_ic.to(device)
        self.X_bc_left = self.X_bc_left.to(device)
        self.X_bc_right = self.X_bc_right.to(device)
        self.X_colloc = self.X_colloc.to(device)

    def generate_training_data(self):
        # Initial condition points: u(x,0) = -sin(πx)
        x_ic = torch.linspace(x_min, x_max, 100).reshape(-1, 1)
        t_ic = torch.zeros_like(x_ic)
        self.X_ic = torch.cat((x_ic, t_ic), dim=1)
        self.u_ic = torch.sin(np.pi * x_ic)

        # Boundary condition points
        t_bc = torch.linspace(t_min, t_max, 100).reshape(-1, 1)
        x_bc_left = torch.zeros_like(t_bc)
        x_bc_right = torch.ones_like(t_bc) * x_max
        self.X_bc_left = torch.cat((x_bc_left, t_bc), dim=1)
        self.X_bc_right = torch.cat((x_bc_right, t_bc), dim=1)

        # Collocation points for PDE
        n_colloc = 10000
        x_colloc = torch.rand(n_colloc, 1) * (x_max - x_min) + x_min
        t_colloc = torch.rand(n_colloc, 1) * (t_max - t_min) + t_min
        self.X_colloc = torch.cat((x_colloc, t_colloc), dim=1)

    def compute_pde_residual(self, x_colloc):
        x_colloc.requires_grad = True
        # u should be the result of  t * Network + sin(pi*x)
        # This is to ensure the network predicts the solution at t=0 and x=0 and x=2 correctly
        # get the vector of t_colloc from x_colloc
        tc_colloc = x_colloc[:, 1:2]  # Extract time component
        xc_colloc = x_colloc[:, 0:1]  # Extract space component

        u = tc_colloc * (xc_colloc - x_max) * (xc_colloc - x_min) * self.network(x_colloc) + torch.sin(np.pi * xc_colloc)

        # Calculate derivatives
        u_grad = torch.autograd.grad(u.sum(), x_colloc, create_graph=True)[0]
        u_t = u_grad[:, 1:2]  # ∂u/∂t
        u_x = u_grad[:, 0:1]  # ∂u/∂x

        # Second derivative ∂²u/∂x²
        u_xx = torch.autograd.grad(u_x.sum(), x_colloc, create_graph=True)[0][:, 0:1]

        # PDE residual: ∂u/∂t + u*∂u/∂x - ν*∂²u/∂x²
        residual = u_t + u * u_x - viscosity * u_xx

        return residual

    def loss_pde(self, x):
        residual = self.compute_pde_residual(x)
        return torch.mean(residual ** 2)

    def train(self, epochs=10000, learning_rate=0.001):
        optimizer = torch.optim.Adam(self.network.parameters(), lr=learning_rate)
        scheduler = ExponentialLR(optimizer, gamma=0.9999)

        history = {'total_loss': [], 'ic_loss': [], 'bc_loss': [], 'pde_loss': []}

        for epoch in range(epochs):
            optimizer.zero_grad()

            # Compute losses
            pde_loss = self.loss_pde(self.X_colloc)

            # Total loss
            total_loss = pde_loss

            # Backpropagation
            total_loss.backward()
            optimizer.step()
            scheduler.step()

            # Store losses
            if epoch % 1000 == 0:
                history['total_loss'].append(total_loss.item())
                history['pde_loss'].append(pde_loss.item())
                print(f"Epoch {epoch}: Loss = {total_loss.item():.6f}")

        return history

### Train PINN for Viscous problem

Note that we need a new `predict_solution_bc` function for the boundary condition case, this function will handle the specific boundary conditions for the viscous Burgers equation.

Because our network predicts directly not the full solution but the residuals we need to adjust the prediction to account for the boundary conditions.

In [ ]:
def predict_solution_bc(solver, x, t):
    """Generate predictions for given space-time points."""
    if not torch.is_tensor(x):
        x = torch.tensor(x, dtype=torch.float32)
    if not torch.is_tensor(t):
        t = torch.tensor(t, dtype=torch.float32)

    x = x.reshape(-1, 1).to(solver.device)
    t = t.reshape(-1, 1).to(solver.device)

    X, T = torch.meshgrid(x.squeeze(), t.squeeze(), indexing='ij')
    X_pred = torch.stack([X.flatten(), T.flatten()], dim=1).to(solver.device)

    solver.network.eval()
    with torch.no_grad():
        tc_colloc = X_pred[:, 1:2]  # Extract time component
        xc_colloc = X_pred[:, 0:1]  # Extract space component

        u_pred = tc_colloc * (xc_colloc - x_max) * (xc_colloc - x_min) * solver.network(X_pred) + torch.sin(np.pi * xc_colloc)
    return u_pred.reshape(x.shape[0], t.shape[0]).cpu().numpy()


# Train and plot the Viscous Burgers Equation solution
x = np.linspace(x_min, x_max, 100)
t = np.linspace(t_min, t_max, 100)

# Train the model
network_viscous = BurgersPINN().to(device)
solver_viscous = ViscousBurgersSolverBC(network_viscous, device)
history_viscous = solver_viscous.train()

# Get PINN solution
u_pinn = predict_solution_bc(solver_viscous, x, t)

In [ ]:
# Compare with reference solution
plot_comparison(x, t, u_pinn, x_ref, t_ref, u_ref)

# Plot training history
plt.figure(figsize=(10, 5))
plt.semilogy(range(0, len(history_viscous['total_loss'])*100, 100), history_viscous['total_loss'], label='Total Loss')
plt.semilogy(range(0, len(history_viscous['pde_loss'])*100, 100), history_viscous['pde_loss'], label='PDE Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training History')
plt.legend()
plt.grid(True)
plt.savefig('./plots/training_history.png')
plt.show()

## 5. <font color='red'>Exercise</font>: Implement Inviscid Burgers Equation

Now it's your turn! Implement the PINN solver for the inviscid Burgers equation. The main differences from the viscous case are:

1. No viscosity term in the PDE (ν = 0)
2. Different handling of shock formation
3. Possible need for more collocation points to handle the discontinuity at the shock
4. Possible need to handle the boundary conditions exactly

Here's a template to get you started. Complete the `compute_pde_residual` method and any other modifications you think are necessary.

In [ ]:
class InviscidBurgersSolver:
    def __init__(self, network, device='cpu'):
        self.network = network.to(device)
        self.device = device
        self.generate_training_data()

        # Move data to device
        self.X_ic = self.X_ic.to(device)
        self.u_ic = self.u_ic.to(device)
        self.X_bc_left = self.X_bc_left.to(device)
        self.X_bc_right = self.X_bc_right.to(device)
        self.X_colloc = self.X_colloc.to(device)

    def generate_training_data(self):
        # TODO: Modify if needed for inviscid case
        # Initial condition points
        x_ic = torch.linspace(x_min, x_max, 100).reshape(-1, 1)
        t_ic = torch.zeros_like(x_ic)
        self.X_ic = torch.cat((x_ic, t_ic), dim=1)
        self.u_ic = torch.sin(np.pi * x_ic)

        # Boundary conditions
        t_bc = torch.linspace(t_min, t_max, 100).reshape(-1, 1)
        x_bc_left = torch.zeros_like(t_bc)
        x_bc_right = torch.ones_like(t_bc) * x_max
        self.X_bc_left = torch.cat((x_bc_left, t_bc), dim=1)
        self.X_bc_right = torch.cat((x_bc_right, t_bc), dim=1)

        # Collocation points
        n_colloc = 10000
        x_colloc = torch.rand(n_colloc, 1) * (x_max - x_min) + x_min
        t_colloc = torch.rand(n_colloc, 1) * (t_max - t_min) + t_min
        self.X_colloc = torch.cat((x_colloc, t_colloc), dim=1)

    def compute_pde_residual(self, x_colloc):
        # TODO: Implement the PDE residual for inviscid Burgers equation
        # Hint: You need to compute ∂u/∂t + u*∂u/∂x = 0

        # TODO: Implement the residual
        residual = None  # Replace this line

        return residual

    # TODO: Implement any additional loss terms needed

    def train(self, epochs=10000, learning_rate=0.001):
        # TODO: Modify the training loop if needed
        pass  # Replace this with your implementation

## 5. Training and Comparison Functions

Here are helper functions to train both models and compare their results. After implementing the inviscid solver, use these functions to analyze the differences between the two solutions.

In [ ]:
def predict_solution(solver, x, t):
    """Generate predictions for given space-time points."""
    if not torch.is_tensor(x):
        x = torch.tensor(x, dtype=torch.float32)
    if not torch.is_tensor(t):
        t = torch.tensor(t, dtype=torch.float32)

    x = x.reshape(-1, 1).to(solver.device)
    t = t.reshape(-1, 1).to(solver.device)

    X, T = torch.meshgrid(x.squeeze(), t.squeeze(), indexing='ij')
    X_pred = torch.stack([X.flatten(), T.flatten()], dim=1).to(solver.device)

    solver.network.eval()
    with torch.no_grad():
        u_pred = solver.network(X_pred)

    return u_pred.reshape(x.shape[0], t.shape[0]).cpu().numpy()

def plot_solutions(x, t, u_viscous, u_inviscid):
    """Plot and compare viscous and inviscid solutions."""
    # Create time slices plot
    plt.figure(figsize=(15, 5))
    times = [0.0, 0.25, 0.48]
    time_indices = [np.abs(t - time).argmin() for time in times]

    for i, (time, idx) in enumerate(zip(times, time_indices)):
        plt.subplot(1, 3, i+1)
        plt.plot(x, u_viscous[:, idx], 'r-', label='Viscous')
        plt.plot(x, u_inviscid[:, idx], 'b--', label='Inviscid')
        plt.xlabel('x')
        plt.ylabel('u')
        plt.title(f't = {time:.2f}')
        plt.grid(True)
        plt.legend()

    plt.tight_layout()
    plt.savefig('./plots-tutorial/comparison_slices.png')
    plt.close()

    # Create contour plots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    X, T = np.meshgrid(t, x)

    c1 = ax1.contourf(T, X, u_viscous, levels=50, cmap='rainbow')
    plt.colorbar(c1, ax=ax1)
    ax1.set_xlabel('t')
    ax1.set_ylabel('x')
    ax1.set_title('Viscous Solution')

    c2 = ax2.contourf(T, X, u_inviscid, levels=50, cmap='rainbow')
    plt.colorbar(c2, ax=ax2)
    ax2.set_xlabel('t')
    ax2.set_ylabel('x')
    ax2.set_title('Inviscid Solution')

    plt.tight_layout()
    plt.savefig('./plots-tutorial/comparison_contours.png')
    plt.close()

## 6. Example Usage and Analysis

Here's an example of how to use the completed implementation. After implementing the inviscid solver, run this code to compare the solutions.

### Expected Observations:
1. The viscous solution will be smooth, while the inviscid solution may develop shocks
2. The inviscid case will show steeper gradients and more abrupt changes
3. The viscous term acts to smooth out discontinuities

Try different initial conditions and parameter values to explore the behavior of both equations.

In [ ]:
# Generate space-time grid for predictions
x = np.linspace(x_min, x_max, 100)
t = np.linspace(t_min, t_max, 100)

# Train viscous solver
network_viscous = BurgersPINN().to(device)
solver_viscous = ViscousBurgersSolver(network_viscous, device)
history_viscous = solver_viscous.train()

# TODO: Uncomment after implementing InviscidBurgersSolver
"""
# Train inviscid solver
network_inviscid = BurgersPINN().to(device)
solver_inviscid = InviscidBurgersSolver(network_inviscid, device)
history_inviscid = solver_inviscid.train()

# Generate predictions
u_viscous = predict_solution(solver_viscous, x, t)
u_inviscid = predict_solution(solver_inviscid, x, t)

# Plot and compare solutions
plot_solutions(x, t, u_viscous, u_inviscid)
"""

# For now, just plot viscous solution
plt.figure(figsize=(10, 6))
u_viscous = predict_solution(solver_viscous, x, t)
X, T = np.meshgrid(t, x)
plt.contourf(T, X, u_viscous, levels=50, cmap='rainbow')
plt.colorbar(label='u')
plt.xlabel('t')
plt.ylabel('x')
plt.title('Viscous Burgers Equation Solution')
plt.savefig('./plots-tutorial/viscous_solution.png')
plt.close()

Finally, let's compare our predicted solution for the inviscid case against finite-difference data.

In [ ]:
# Load inviscid reference data from FD solution
# This is a much larger file, so it will take longer to upload!
# We also need to take a sample of the data since it's unlikely to fit in memory.
rng = np.random.default_rng()
inviscid_fd_data = np.load('burgers_fd_data_inviscid.npz')

# Assume u shape is (len(x), len(t))
inviscid_x_ref = inviscid_fd_data['x']
inviscid_t_ref = inviscid_fd_data['t']
inviscid_u_ref = inviscid_fd_data['u']

# Define subsampling step
subsample_step_x = 4  # e.g., every 4th x-point
subsample_step_t = 5  # e.g., every 5th t-point

# Subsample the data
subsampled_x = inviscid_x_ref[::subsample_step_x]
subsampled_t = inviscid_t_ref[::subsample_step_t]
subsampled_u = inviscid_u_ref[::subsample_step_x, ::subsample_step_t]

In [ ]:
plot_comparison(x, t, u_pinn, subsampled_x, subsampled_t, subsampled_u, case="inviscid")